In [ ]:
#一些常用的命令
!nvidia-smi

import tensorflow as tf
tf.__version__
tf.config.list_physical_devices('GPU')

In [ ]:
#一些常用的命令
!git status
!git add .
!git commit -m "correct and optimize some code"
!git push

In [ ]:
from datetime import datetime
timestr= datetime.now()
print(timestr)

!conda info

In [ ]:
!rm -rf tmp*.csv

In [ ]:
%history

In [25]:
print("mainSimpleStep0为简化的步骤0程序，只有分离式的训练，并获得数据")
'''
parser.add_argument('-np','--numEpochs', default=1000, type=int,help='分离式模型每层训练次数')
parser.add_argument('-trn','--trainOrNot', default=1,type=int,help='训练吗?')
parser.add_argument('-ten','--testOrNot', default=1,type=int,help='测试吗?')
parser.add_argument('-tr','--testRatio', default = 0.9,type=float,help='测试集比例')
parser.add_argument('-sm','--sampleMethond', default = 2,type=int,help='重采样方法：1：OverSample,2:SMOTE')
parser.add_argument('-ht','--hType', default = 1,type=int,help='1为经典8层层次，2为拥堵定义的2层层次')
'''


!python3 mainSimpleStep0.py --numEpochs 2000 --trainOrNot 0 --testOrNot 1 --sampleMethond 2 --hType 1 --testRatio 0.95
#!python3 mainSimpleStep0.py --numEpochs 2000 --trainOrNot 1 --testOrNot 1 --sampleMethond 2 --hType 2 --testRatio 0.95

mainSimpleStep0为简化的步骤0程序，只有分离式的训练，并获得数据
0.主程序开始，建立多层嵌套决策树模型，3080ti的GPU是AMD2400CPU 运算速度100倍
0.这是简化程序，原始带有更多测试和原始模型的程序在mainTestCSVMLP3(hmcnf_keras).ipynb
程序编号为0，注意这是mainSimpleStep0为简化的步骤0程序
2024-05-05 10:32:41.370860: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.

2slot的数据列表为：headName2SlotXY96
 ['vehID', 'redLightTime', 'distToRedLight', 'speed', 'laneAvgSpeed', 'arriveTime1', 'arriveTime2', 'vehLaneID', 'ArrTimeDivRedTime', 'vehPos_1', 'vehSpeed_1', 'vehPos_2', 'vehSpeed_2', 'vehPos_3', 'vehSpeed_3', 'vehPos_4', 'vehSpeed_4', 'vehPos_5', 'vehSpeed_5', 'vehPos_6', 'vehSpeed_6', 'vehPos_7', 'vehSpeed_7', 'vehPos_8', 'vehSpeed_8', 'vehPos_9', 'vehSpeed_9', 'vehPos_10', 'vehSpeed_10', 'vehPos_11', 'vehSpeed_11', 'vehPos_12', 'vehSpeed_12', 'vehPos_13', 'vehSpeed_13', 'vehPos_14', 'vehSpeed_14', 'vehPos_15', 'vehSpeed_15', 'vehPos_16', 'vehSpeed_16', 'vehPos_17', 'vehSpeed_17', 'vehPos_18', 'veh

In [27]:
print("mainSimpleStep1为简化的步骤1程序，用于获得低概率样本")
!python3 mainSimpleStep1.py --level 0 1 2 3 4 5 6 7 --threshold 0.7

mainSimpleStep1为简化的步骤1程序，用于获得低概率样本
1.接编号为0的主程序,先找出低概率样本，
2.对较低概率的样本进行蒙特卡洛模拟分析，原始对应程序为mainSimSumoFranceDatra
3.最终进行分析，程序编号为1
3.最终进行分析，有简化程序mainSimpleStep1.py
2024-05-05 10:39:05.400540: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.
1.1 主程序开始
2024-05-05 10:39:10.108942: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-05-05 10:39:10.133546: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:968] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 10:39:10.133614: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.725GHz coreCount: 30 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 312.97GiB/s
2024-0

In [ ]:
########################################################################################################################
print("接程序1: 对较低概率的样本进行蒙特卡洛模拟分析,原始对应程序为mainSimSumoFranceDatra")
print("因为配置失误，采用将低概率的样本进行保存为文件，然后再root用户下命令行模式用SUMO模拟（不使用conda）")
print("输出为sumoSimData？？？.csv,里面有每个样本的sumo输出，kerasNN输出以及原始的输入输出")
print("程序编号为2")
print("程序编号为2")
########################################################################################################################
#!python3 sumoSimByFrance.py#运行runSumoSimFun.py 中test3
#--sectionNum 5 一般为电脑CPU的物理核心数，太大会导致电脑忙死，无法运行其他程序

# sectionNum 8,simNum = 150,8hour,
!python3 mainSimpleStep2_sumoSimByFranceParallSim1.py --testMode 1 --sectionNum 8 --level 7 --simNum 150  --numRunSample 100000 --makeOneFileOrNot 1

# sectionNum 16,simNum = 15,38min,

#python3 mainSimpleStep2_sumoSimByFranceParallSim1.py --testMode 1 --sectionNum 10 --level 7 --simNum 10  --numRunSample 1000 --makeOneFileOrNot 1 #测试用

In [ ]:
'''step0获得:原始数据库数据（有一定挑选）'''
'''step1获得:根据step0的识别模型和识别结果的从step0中样本挑选低概率样本xlowpra'''
'''step2获得：step1中低概率样本中SUMO成果仿真获取的样本（有一定挑选，去掉无法仿真样本大概1%）'''
'''step3获得：将step2的样本与对应的step1的低概率样本的进行合成，并训练加强模型stage2Mode'''                                                 
'''step3找到step2样本（就是df_step2）在step1的样本（xlowpra1）相对应的样本xlowpra2。然后df_step2和xlowpra2合成为x,用于训练加强模型stage2Mode''' 

!python3 mainSimpleStep3.py --level 7 --simNum 150  --step0ModeCheck 1 --numEpochs 2000

In [ ]:
print("程序4b: 在论文中体现，不同标记下的不同最大速度下的最小速度分布（不停车过路口）,获得一般参数下的预测值")
print("mainSimpleStep4B,需要用step3中lowprobsample")

!python3 mainSimpleStep4B.py --level 7 --simNum 150  --genSimDataOrNot 0 --nSamples 10000

In [ ]:
print("程序4A: 在论文中体现，实现验证幽灵堵车规则")
print("接程序3:程序3为根据输入的SUMO模拟特征对低概率样本进行重新训练,从而获得识别正确率提升")
print("mainSimpleStep4为简化的步骤4程序，注意只能接mainSimpleStep3的输出结果")
print("基本废弃 幽灵堵车规则")
!python3 mainSimpleStepa4A.py

In [ ]:
print("程序4C: 在论文中体现，实现reactTime参数估计")
print("mainSimpleStep4C注意只能接mainSimpleStep3的输出结果,c参数拟合")
print("基本废弃 reactTime参数估计")
!python3 mainSimpleStep4C.py

In [ ]:
print("程序4D: 在论文中体现，不同标记下的最小速度分布（不停车过路口）,获得优化参数下的预测值")
print("mainSimpleStep4D,需要用step3中lowprobsample,4b中的step4b_analy1_SSVP_level7.csv和4c中的pipeline的预测reactTime")
#root用户下命令行模式用SUMO模拟（不使用conda）
print("基本废弃 程序4D")
!python3 mainSimpleStep4D.py #root用户下命令行模式用SUMO模拟（不使用conda）


In [ ]:
a=1
b = a==1
print(b)

In [ ]:
num1 = [0,0,0,0]#<20=0,<30=1,<40=2,<50=3
analy = [num1 for i in range(7)]
analy

In [ ]:
import numpy as np
a =[[1,2,3],[4,5,6]]
a[1][1] = 2
a

a =np.array(a)
a[1,1] = 33
a

In [ ]:
[0]*9

In [ ]:
import numpy as np
a=np.array([1,2,3,4,5])
aa=np.array([[1,2],[2,1],[3,3,3],[4],[5]])

tmp = a<3
print(tmp)
b = aa[tmp]
b

In [ ]:
a = [[1,2,3],[4,5,6]]
b = [[7,8,9],[11,12,13]]
c = a+b
print(c)

a.extend(b)
print(a)

#a.append(b)
#print(a)

In [ ]:
a = [[1,2,3],[4,5,6]]
b = [[7,8,9],[11,12,13]]
c = a+b
print(c)

a.extend(b)
print(a)

#a.append(b)
#print(a)

In [ ]:

import pandas as pd
A=pd.DataFrame({'a':[1,2,3],'b':[2,3,4],'c':[2,4,5]})
A=pd.DataFrame([[1,2,3],[2,3,4],[2,4,5]])
print(A)
print(A.shape)
print(A.iloc[:,-1])

A= A.drop(labels=[0],axis=0)
print(A)
A=pd.DataFrame({'a':[1,2,3],'b':[2,3,4],'c':[2,4,5]})
print(A)
print(A[['a','b']])

In [ ]:

import pandas as pd
A=pd.DataFrame({'a':[1,2,3],'b':[2,3,4],'c':[2,4,5]})
A=pd.DataFrame([[1,2,3],[2,3,4],[2,4,5]])
print(A)
print(A.shape)
print(A.iloc[:,-1])

A= A.drop(labels=[0],axis=0)
print(A)
A=pd.DataFrame({'a':[1,2,3],'b':[2,3,4],'c':[2,4,5]})
print(A)
print(A[['a','b']])

In [ ]:
sampleIndex

In [ ]:
1. 以何种方式过红灯,如何优化
A.红灯状态下，如果预测minSpeed为高速而且概率较高，那样可以认为车辆能以高速或者一般速度通过前方路口，可以进行模拟,看几个速度可以通过
如果我已经知道我会以较高速度通过路口，不做模拟，直接给出值
B.红灯状态下，如果预测minSpeed为低速而且概率较高，那样可以认为车辆一般无法不停车通过路口
如果我已经知道我会以较低度通过路口，不做模拟，直接给出值
C.中间状态呢，红灯状态下如果预测minSpeed为高速而且概率较低，可以进行模拟,看几个速度可以通过
D.中间状态呢，红灯状态下如果预测minSpeed为低速而且概率较低，可以进行模拟,看几个速度可以通过

2.预测幽灵堵车
原因式本来这条道路机容易出现幽灵堵车，和有车不停变道

A.如果蒙特卡洛预测能高速通过路口，但是预测minSpeed比较低，同时如果设定车辆延迟比较高时，也不能通过路口，认为出现幽灵堵车
B.如果初始模型能预测通过路口，但是加入道路特征和设定车辆延迟比较高时也不能通过路口，也不能通过路口，认为出现幽灵堵车
C.如果初始模型能预测能通过路口，但是概率比较低，但是加入道路特征和设定车辆延迟比较高时不能通过路口，认为出现幽灵堵车
D.如果初始模型和蒙特卡洛模拟预测都能通过路口，但是概率比较低，但是加入道路特征和设定车辆延迟比较高时不能通过路口，认为出现幽灵堵车
E.如果初始模型和蒙特卡洛模拟预测都能通过路口，但是概率比较高，但是加入道路特征和设定车辆延迟比较高时不能通过路口，不知道。

In [ ]:

from numpy import random
a = random.normal(0.7,0.2)
a
from numpy import random
b = random.uniform(1.7,1.9)
b
c =[[20,30]/3.5,[30,40],[40,50],[50,60],[60,70]]/3.6
c

In [ ]:

a = np.array([[1,2],[3,4]])
b = np.array([[1,2]])
print(a)
print(b)
c = np.concatenate((a,b),axis = 0)
print(c)

c = np.empty([5,3]).tolist()
c[1][1] = [1,2,3]
print(c)

c = np.zeros([5,3]).tolist()
c[1][1] = [1,2,3]
print(c)

In [ ]:
a = list(range(20))
s1 = 2
s2 = 10
minSpeedList1 = [num for num in  a if num > s1 and num <s2]
minSpeedList1

In [ ]:
from multiprocessing import Process
import time


g1 = [0]*9
g1[4] =3
def my_function(x):
    #global g1
    
    time.sleep(4)
    g1[x] = 1
    print(g1)
    print(f'处理 {x}')
    


process = Process(target=my_function, args=(2,))
process.start()  # 在一个新的进程中开始执行my_function
process.join()

print("进程完成",g1)


In [ ]:
import multiprocessing
 
def my_function(arg1, arg2):
    print(arg1,arg2)
    result = arg1*arg2
    return result
 
if __name__ == '__main__':
    pool = multiprocessing.Pool()
    
    args = [(1, 2), (3,4)]  # 定义参数列表
    
    results = pool.starmap(my_function, args)   # 调用map函数进行并发计算
    
    print(results)

In [ ]:
import multiprocessing as mp
params = dict()
params[0] = 2
params[1] = 3
def job1(x,params):
    t= x*params[0]*params[1]
    return t

def job0(z):
    x = z[0]
    params = z[1]
    return job1(x,params)

def multicore():
    numSamples = 10
    data_list=[[i,params] for i in range(numSamples)]
    pool = mp.Pool() # 无参数时，使用所有cpu核
    # pool = mp.Pool(processes=3) # 有参数时，使用CPU核数量为3
    res = pool.map(job0, data_list)
    print(res)
    

multicore()




In [ ]:
%cd /home/liuli/myCodes/SUMONBDT/branch_2slot_9label/data/
%pwd
!rm *.txt
!rm step4D*.pkf

In [ ]:
maxSpeedList =[[10,15],[20,25],[30,35],[40,80]]
a= len(maxSpeedList)
print(a)

In [ ]:
a = 3.1456
b = "#%.3f_%.5f#" %(a,a)
print(b)

In [ ]:
import numpy as np

record1 = np.zeros((9, 5)).tolist()

for i in range(9):
    record1 [i][1] = ['1',3,5]
    
record1

In [ ]:
from numpy import percentile 

In [ ]:
[0,1,3,4,5]*7.7